In [ ]:
import copy
from numpy import asarray
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
import random
import math
from torch.autograd import Variable
from torch.nn.functional import kl_div, softmax, log_softmax
import torch.optim as optim
import torch.utils.data as data
from os.path import exists, join, split
from os import listdir
from os.path import join
from PIL import Image, ImageFilter , ImageDraw
import PIL
import random
import matplotlib.pyplot as plt
#! pip install madgrad
#! pip install efficientnet_pytorch

In [ ]:
# Hyperparameters

num_class = 10 # Number of classes
seed=111 # Seed for the algorithm
batch_size = 32
num_train = 100 # Number of training image by classe
cutout=16  # Parameter for the cutout
num_epochs=20
valid_size = 200 # Validation set size
lr=0.001 # Learning rate

In [ ]:
class Dataset_sub_CIFAR(data.Dataset):

    def __init__(self, data_feature, data_target,transform,phase='label'):
        self.data_feature = data_feature
        self.data_target = data_target
        self.transform = transform
        self.phase=phase

    def __len__(self):
        return len(self.data_feature)

    def __getitem__(self, index):
        if self.phase=='label':
            data_feature = self.transform(Image.fromarray(np.uint8(self.data_feature[index])))
            data_target =  self.data_target[index]
            return data_feature, data_target

        else:
            data_feature = self.data_feature[index].float()
            return data_feature

no_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])

transform_test = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])

weak_augmentation = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomHorizontalFlip(p=0.5),
])

strong_augmentation = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(size=128,padding=int(128*0.125),padding_mode='reflect'),
])


#Dataset loading
CIFAR10_train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=None, download=True)
CIFAR10_test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=None, download=True)
np.random.seed(seed=seed)
permuation=np.random.permutation(len(CIFAR10_train_dataset.targets))

Original_train_data_x = (CIFAR10_train_dataset.data)
Original_train_data_y = np.array(CIFAR10_train_dataset.targets)
Original_train_data_x = Original_train_data_x[permuation]
Original_train_data_y = Original_train_data_y[permuation]

Original_test_data_x = CIFAR10_test_dataset.data
Original_test_data_y = np.array(CIFAR10_test_dataset.targets)

incr_class = torch.zeros(num_class)
train_idx_dico = {} #labeled images index dictionnary

for i in range(num_class):
    train_idx_dico[str(i)] = []

valid_idx = np.zeros(num_class * valid_size, dtype=np.int32) #validation images indexes (2000)
incr_t = 0
incr_v = 0
incrtotal = 0

for idx in range(len(Original_train_data_y)):
    class_y = Original_train_data_y[idx]
    incrtotal += 1

    train_idx_dico[str(class_y)].append(idx)
    incr_class[class_y] += 1 #count the number of image per class
    incr_t += 1


train_idx = np.zeros(num_class * num_train, dtype=np.int32) #train labeled images indexes (1000)
list_train_id = []
list_unalabel_id = []
valid_idx = []
unlabel_idx_dico = {}
for i in range(num_class):
    unlabel_idx_dico[str(i)] = []
for i in range(num_class):
    list_train_id = list_train_id + train_idx_dico[str(i)][0:num_train]
    valid_idx =valid_idx + train_idx_dico[str(i)][num_train:num_train+valid_size]
    list_unalabel_id = list_unalabel_id + train_idx_dico[str(i)][num_train+valid_size::]
    unlabel_idx_dico[str(i)] = train_idx_dico[str(i)][num_train::]

#Get labeled and unlabeled data

x_train = Original_train_data_x[[int(i) for i in list_train_id]]
y_train = Original_train_data_y[[int(i) for i in list_train_id]]

x_unlabeled = Original_train_data_x[[int(i) for i in list_unalabel_id]]
y_unlabeled = Original_train_data_y[[int(i) for i in list_unalabel_id]]

#Get validation set data
x_valid = Original_train_data_x[[int(i) for i in valid_idx]]
y_valid = Original_train_data_y[[int(i) for i in valid_idx]]

# Printing the size of the training, validation and test sets
print('Semi-supervised training')
print('Number of training examples: ' + str(x_train.shape[0]))
print('Number of unlabeled examples: ' + str(x_unlabeled.shape[0]))
print('Number of validation examples: ' + str(x_valid.shape[0]))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#Dataloader creation

test_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(Original_test_data_x, Original_test_data_y, transform=transform_test),
    batch_size = batch_size,
    shuffle=False, num_workers=2)

valid_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_valid, y_valid, transform=transform_test),
    batch_size=batch_size,
    shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 1000
Number of unlabeled examples: 47000
Number of validation examples: 2000
mu = 47


In [ ]:
criterion = nn.CrossEntropyLoss()

def learning_rate_scheduler(init, k):
    return init #Version that does not change with respect to epoch
    optim_factor = 0
    if(k > 20):
        optim_factor = 5
    elif(k > 15):
        optim_factor = 4
    elif(k > 10):
        optim_factor = 3
    elif(k > 7):
        optim_factor = 2
    elif(k > 5):
        optim_factor = 1

    return init*math.pow(0.1, optim_factor)


# Training
def trainSupervised(epoch,net,trainloader,k):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    print('\n=> Training Epoch #%d, LR=%.4f, k=%.d' %(epoch, learning_rate_scheduler(lr, k),k))
    optimizer = optim.SGD(net.parameters(), lr=learning_rate_scheduler(lr, k), momentum=0.9, weight_decay=5e-4,nesterov=True)
    for batch_idx, (inputs, targets) in enumerate(trainloader):

      inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
      optimizer.zero_grad()
      inputs, targets = Variable(inputs), Variable(targets)
      outputs = net(strong_augmentation(inputs)) # Forward Propagation
      loss = criterion(outputs, targets)  # Loss
      loss.backward()  # Backward Propagation
      optimizer.step() # Optimizer update

      train_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += predicted.eq(targets.data).cpu().sum()
      if batch_idx % 15 == 0:
        print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%' %(epoch, num_epochs, batch_idx+1, (len(trainloader.dataset)//batch_size)+1, loss.item(), 100.*correct/total))

def test(epoch,net,test_loader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%\nBest accuracy yet : %.2f%%" %(epoch, loss.item(), acc, best_acc))
    return acc

In [ ]:
best_acc=0
weights = models.ResNet18_Weights.DEFAULT

net = models.resnet18(weights=weights)
net_save = models.resnet18(weights=weights)

net.fc = nn.Linear(net.fc.in_features, num_class) # Setting the number of output neurones to the number of different classes
net.fc = net.fc.cuda()

net_save.fc = nn.Linear(net_save.fc.in_features, num_class)
net_save.fc = net_save.fc.cuda()

net = net.cuda()
net_save = net_save.cuda()

tau=0.98 # threshold for fixmatch

train_loader = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_train, y_train, transform=no_transform),
    batch_size=batch_size,shuffle=True, num_workers=2)

train_loader_unlabel = torch.utils.data.DataLoader(
    Dataset_sub_CIFAR(x_unlabeled, y_unlabeled, transform=no_transform),
    batch_size=batch_size,shuffle=False, num_workers=2)

epochs=[]
accuracies=[]

def trainNetwork(net,train_loader,k):
  global best_acc
  num_epochs=20
  for epoch in range(num_epochs):
    trainSupervised(epoch,net,train_loader,k)
    acc=test(epoch,net,valid_loader)
    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%\n' %(acc))
        net_save.load_state_dict(net.state_dict(), strict=True)
        best_acc=acc
    epochs.append(epoch+num_epochs*k)
    accuracies.append(best_acc)

indexDone=[]

for k in range(3):
  #Supervised Training
  trainNetwork(net,train_loader,k)

  #Pseudo-labelization

  print('=> Pseudo-labelization')
  for batch_idx, (inputsCPU, targetsCPU) in enumerate(train_loader_unlabel):
    net.eval()
    inputs, targets = inputsCPU.cuda(), targetsCPU.cuda()

    out1 = net(weak_augmentation(inputs))
    pseudo_label1 = torch.softmax(out1, dim=-1)
    max_probs1, targets_u1 = torch.max(pseudo_label1, dim=-1)
    max_probs1 = max_probs1.cpu().detach().numpy()
    targets_u1 = torch.tensor(targets_u1.cpu().detach().numpy())
    newTarget1 = np.array(targets_u1)

    out2 = net(weak_augmentation(inputs))
    pseudo_label2 = torch.softmax(out2, dim=-1)
    max_probs2, targets_u2 = torch.max(pseudo_label2, dim=-1)
    max_probs2 = max_probs2.cpu().detach().numpy()
    targets_u2 = torch.tensor(targets_u2.cpu().detach().numpy())
    newTarget2 = np.array(targets_u2)

    out3 = net(weak_augmentation(inputs))
    pseudo_label3 = torch.softmax(out3, dim=-1)
    max_probs3, targets_u3 = torch.max(pseudo_label3, dim=-1)
    max_probs3 = max_probs3.cpu().detach().numpy()
    targets_u3 = torch.tensor(targets_u3.cpu().detach().numpy())
    newTarget3 = np.array(targets_u3)

    for j in range(len(max_probs1)):
      if max_probs1[j]>tau and max_probs2[j]>tau and max_probs3[j]>tau and newTarget1[j]==newTarget2[j] and newTarget1[j]==newTarget3[j]:
        imageIndex = batch_idx*batch_size + j
        newInput = x_unlabeled[imageIndex,:,:,:]
        
        if not (imageIndex in indexDone):
          x_train = np.append(x_train,[newInput],0)
          y_train = np.append(y_train,newTarget1[j])
          indexDone.append(imageIndex)

    if batch_idx % 100 == 0:
        print('| Iter[%4d/%4d] New label : %3d/%3d possible' %(batch_idx+1,x_unlabeled.shape[0]//batch_size,len(indexDone),x_unlabeled.shape[0]))

  train_loader = torch.utils.data.DataLoader(
  Dataset_sub_CIFAR(x_train, y_train, transform=no_transform),
  batch_size=batch_size,shuffle=True, num_workers=2)



=> Training Epoch #0, LR=0.0010, k=0
| Epoch [  0/ 20] Iter[  1/ 32]		Loss: 2.5696 Acc@1: 12.500%
| Epoch [  0/ 20] Iter[ 16/ 32]		Loss: 1.9422 Acc@1: 17.773%
| Epoch [  0/ 20] Iter[ 31/ 32]		Loss: 1.3338 Acc@1: 30.645%

| Validation Epoch #0			Loss: 0.6322 Acc@1: 55.85%
Best accuracy yet : 0.00%
| Saving Best model...			Top1 = 55.85%


=> Training Epoch #1, LR=0.0010, k=0
| Epoch [  1/ 20] Iter[  1/ 32]		Loss: 1.2732 Acc@1: 65.625%
| Epoch [  1/ 20] Iter[ 16/ 32]		Loss: 1.2038 Acc@1: 63.477%
| Epoch [  1/ 20] Iter[ 31/ 32]		Loss: 0.8799 Acc@1: 66.230%

| Validation Epoch #1			Loss: 0.3482 Acc@1: 69.45%
Best accuracy yet : 55.85%
| Saving Best model...			Top1 = 69.45%


=> Training Epoch #2, LR=0.0010, k=0
| Epoch [  2/ 20] Iter[  1/ 32]		Loss: 0.6825 Acc@1: 81.250%
| Epoch [  2/ 20] Iter[ 16/ 32]		Loss: 0.7670 Acc@1: 78.906%
| Epoch [  2/ 20] Iter[ 31/ 32]		Loss: 0.8438 Acc@1: 78.427%


KeyboardInterrupt: ignored

In [ ]:
def test_final(net,testloader):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = Variable(inputs), Variable(targets)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()
            if batch_idx == 0:
                predicted_concat = predicted.clone()
            else:
                predicted_concat = torch.cat((predicted_concat, predicted), 0)

        # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| TEST \tLoss: %.4f Acc@1: %.2f%%" %( loss.item(), acc))
    return predicted_concat.cpu().numpy()
    
#redicted_concat = test_final(net,test_loader)
#predicted_concat = test_final(net_save,test_loader)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(epochs,accuracies)